In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_wine

# 1 Memuat Dataset

In [2]:
data = load_wine()
X, y = data.data, data.target

In [3]:
# Mengubah menjadi DataFrame untuk analisis yang lebih mudah
df = pd.DataFrame(X, columns=data.feature_names)
df['target'] = y
df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


Dataset ini memiliki 178 baris dan 13 kolom fitur independen serta satu kolom dependen (target). Masih ingatkah tahapan terakhir persiapan data sebelum melatih model? Yup! Anda perlu membagi dataset menjadi data latih dan data uji. Pada kasus ini, kita akan membagi ukuran dataset menjadi 80% data uji dan 20% data latih. Silakan eksplorasi mandiri untuk proporsi lainnya ya.

In [4]:
# membagi data'
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Filter Methods

In [6]:
# Menggunakan SelectKBest untuk memilih fitur terbaik
filter_selector = SelectKBest(score_func=chi2, k=2) # memulih 2 fitur terbaik
X_train_filter = filter_selector.fit_transform(X_train, y_train)
X_test_filter = filter_selector.transform(X_test)

print("fitur yang dipilih dengan Filter Method (SelectKBest):")
print(filter_selector.get_support(indices=True))


fitur yang dipilih dengan Filter Method (SelectKBest):
[ 9 12]


SelectKBest
- SelectKBest adalah salah satu metode dari filter methods yang umum digunakan untuk memilih fitur terbaik berdasarkan skor statistik tertentu.

  - Parameter score_func=chi2 berarti metode ini menggunakan Chi-squared (χ²) sebagai fungsi skor. Chi-squared adalah metode uji statistik yang digunakan untuk mengukur independensi antara dua variabel kategorikal.
  - Parameter k=2 artinya kita hanya akan memilih 2 fitur terbaik dari semua fitur yang ada di dataset. Dengan kata lain, ini akan menyeleksi dua fitur dengan skor tertinggi berdasarkan hasil uji Chi-squared.

  Kedua parameter ini bisa Anda sesuaikan dengan studi kasus dan trial and error ya. 



- fit_transform(X_train, y_train)
  - fit(X_train, y_train) merupakan metode yang akan menghitung skor Chi-squared untuk setiap fitur pada data pelatihan (X_train) terhadap target (y_train).
  - transform(X_train) merupakan fungsi yang bertugas untuk menyalin data dari X_train, tetapi pada kasus ini hanya memiliki fitur yang tersimpan pada filter_selector.

Hasilnya adalah sebuah dataset pelatihan baru (X_train_filter) yang hanya mengandung dua fitur terbaik hasil perhitungan Chi-squared.

- transform(X_test)
Fungsi transform(X_test) digunakan untuk mentransformasikan dataset uji (X_test) dengan memilih fitur yang sama yang dipilih dari dataset pelatihan.
- get_support(indices=True)
Fungsi get_support(indices=True) digunakan untuk mendapatkan indeks dari fitur-fitur yang dipilih. Dalam hal ini, fitur yang dipilih adalah dua fitur dengan skor tertinggi berdasarkan uji Chi-squared

Fungsi ini akan mencetak indeks fitur yang dipilih dari dataset awal yang bisa digunakan untuk memahami fitur mana yang dipertahankan setelah proses seleksi. Sehingga, output akhir dari kode ini merupakan dataset yang sudah difilter dan informasi angka indeks dari fitur yang dipilih.

Sampai di sini, Anda sudah memiliki sebuah dataset yang terdiri dari dua buah fitur dengan skor independensi paling besar. Tahan sejenak rasa ingin tahu Anda, karena pada akhir materi ini kita akan membandingkan performa dari ketiga metode feature selection.



# 3. Wrapper Methods

In [7]:
# Menggunakan RFE (Recursive Feature Elimination) dengan Logistic Regression

model = LogisticRegression(solver='lbfgs', max_iter=5000)
rfe_selector = RFE(model, n_features_to_select=2) # memilih 2 fitur terbaik
X_train_rfe = rfe_selector.fit_transform(X_train, y_train)
X_test_rfe = rfe_selector.transform(X_test)

print("fitur yang dipilih dengan Wrapper Method (RFE):")
print(rfe_selector.get_support(indices=True))

fitur yang dipilih dengan Wrapper Method (RFE):
[0 6]


Mirip dengan filter methods pada materi sebelumnya, kode di atas akan mencari dua buah fitur yang paling relevan berdasarkan perhitungan Recursive Feature Elimination. Namun, metode ini menghasilkan indeks fitur yang berbeda jika kita bandingkan dengan filter methods.

Mengapa hal ini bisa terjadi? Tenang saja ini merupakan hal yang wajar, perbedaan ini disebabkan kedua metode ini menggunakan perhitungan matematis yang berbeda. Chi2 akan mengukur hubungan antara fitur dan target secara independen tanpa menggunakan model pembelajaran mesin dan tidak mempertimbangkan interaksi antar fitur. Lalu, RFE akan menghitung menggunakan model machine learning dan secara iteratif memilih fitur terbaik.

# 4. Embedded Method

In [8]:
# Menggunakan Random Forest untuk mendapatkan fitur penting
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
 
# Mendapatkan fitur penting
importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1]
 
# Menentukan ambang batas untuk fitur penting
threshold = 0.05  # Misalnya, ambang batas 5%
important_features_indices = [i for i in range(len(importances)) if importances[i] >= threshold]
 
# Memindahkan fitur penting ke variabel baru
X_important = X_train[:, important_features_indices]  # Hanya fitur penting dari data pelatihan
X_test_important = X_test[:, important_features_indices]  # Hanya fitur penting dari data pengujian
 
# Mencetak fitur yang dipilih
print("Fitur yang dipilih dengan Embedded Methods (di atas ambang batas):")
for i in important_features_indices:
    print(f"{data.feature_names[i]}: {importances[i]}")

# X_important sekarang berisi hanya fitur penting
print("\nDimensi data pelatihan dengan fitur penting:", X_important.shape)
print("Dimensi data pengujian dengan fitur penting:", X_test_important.shape)

Fitur yang dipilih dengan Embedded Methods (di atas ambang batas):
alcohol: 0.11239773542143086
flavanoids: 0.20229341635663622
color_intensity: 0.1712021830864957
hue: 0.07089132259413944
od280/od315_of_diluted_wines: 0.1115643167260497
proline: 0.13904586955351153

Dimensi data pelatihan dengan fitur penting: (142, 6)
Dimensi data pengujian dengan fitur penting: (36, 6)


Secara garis besar, kode di atas akan menjalankan beberapa langkah hingga akhirnya mendapatkan fitur yang paling relevan menurut perhitungannya.

Setelah model dilatih, Anda bisa mendapatkan informasi tentang peran fitur menggunakan atribut feature_importances_. Nilai ini menunjukkan seberapa penting setiap fitur dalam membantu model membuat keputusan (pada kasus ini klasifikasi). Fitur dengan nilai feature_importance yang lebih tinggi memiliki pengaruh yang lebih besar terhadap prediksi model.

Untuk menampilkan fitur terpenting, nilai feature_importance perlu diurutkan dari yang tertinggi ke terendah menggunakan np.argsort(importances)[::-1] sehingga Anda akan mendapatkan indeks fitur berdasarkan urutan kontribusinya.

Selanjutnya, Anda perlu menetapkan ambang batas (threshold). Pada kasus ini, kita sepakat menentukan nilai sebesar 0.05 yang artinya kita hanya akan mempertahankan fitur yang memiliki nilai kepentingan di atas 5%. Bagaimana caranya memilih fiturnya? Tugas ini akan dieksekusi oleh baris kode [i for i in range(len(importances)) if importances[i] >= threshold] yang bertugas untuk memilih indeks fitur yang memenuhi kriteria tersebut.

Setelah fitur-fitur penting dipilih, Anda perlu memindahkan data yang berisikan fitur yang dianggap penting ke dalam variabel baru. X_important adalah versi dari X_train yang hanya berisi fitur-fitur yang dianggap penting, dan hal yang sama dilakukan untuk data pengujian (X_test_important).

Kode di atas juga mencetak daftar fitur yang dipilih beserta nilai kontribusinya sehingga kita bisa melihat fitur mana saja yang dianggap signifikan oleh model Random Forest. Terakhir, kode akan mencetak dimensi dari data pelatihan dan pengujian setelah seleksi fitur dengan tujuan memberikan gambaran tentang seberapa banyak fitur yang telah disaring.

# 5. Evaluasi Fitur

In [12]:
# Evaluasi dengan fitur terpilih dari masing-masing metode
def evaluate_model(X_train, X_test, y_train, y_test, model):
    model.fit(X_train, y_train)
    accuracy = model.score(X_test, y_test)
    return accuracy

Kode di atas merupakan sebuah fungsi yang akan melakukan pelatihan dan mendapatkan metriks evaluasi (accuracy) seperti yang sudah Anda biasa lakukan pada modul-modul sebelumnya, tetapi dibungkus pada sebuah fungsi.

Selanjutnya, mari kita latih model machine learning berdasarkan fitur yang telah ditentukan oleh masing-masing metode feature selection menggunakan kode berikut.

In [13]:
# Model Logistic Regression untuk Filter Methods
logistic_model_filter = LogisticRegression(max_iter=200)
accuracy_filter = evaluate_model(X_train_filter, X_test_filter, y_train, y_test, logistic_model_filter)
 
# Model Logistic Regression untuk Wrapper Methods
logistic_model_rfe = LogisticRegression(max_iter=200)
accuracy_rfe = evaluate_model(X_train_rfe, X_test_rfe, y_train, y_test, logistic_model_rfe)
 
# Model Random Forest untuk Embedded Methods
accuracy_rf = evaluate_model(X_important, X_test_important, y_train, y_test, rf_model)

In [14]:
print(f"\nAkurasi Model dengan Filter Methods: {accuracy_filter:.2f}")
print(f"Akurasi Model dengan Wrapper Methods: {accuracy_rfe:.2f}")
print(f"Akurasi Model dengan Embedded Methods: {accuracy_rf:.2f}")


Akurasi Model dengan Filter Methods: 0.89
Akurasi Model dengan Wrapper Methods: 0.94
Akurasi Model dengan Embedded Methods: 1.00


Seperti yang dapat Anda lihat masing-masing metode memiliki performa yang berbeda. Hal ini karena penggunaan fitur yang berbeda pada proses pelatihan model machine learning. Karena embedded methods memiliki akurasi sempurna, mungkin Anda berpikir “Ya sudah aku pakai embedded methods saja untuk semua kasus.” Eiitss, tidak semudah itu kawan, meskipun embedded methods bisa digunakan dalam berbagai skenario, tetapi tidak semua masalah cocok menggunakan metode ini. Ada beberapa hal yang menjadi pertimbangan ketika Anda akan melakukan feature selection.